In [5]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

def main():
    # 1. DB設定
    db_name = 'google_repos.db'
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS repositories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            language TEXT,
            stars TEXT
        )
    ''')
    cursor.execute('DELETE FROM repositories')
    conn.commit()

    print("DB準備完了。強力な検索モードでスクレイピングを開始します...")

    # 2. スクレイピング
    url = "https://github.com/orgs/google/repositories"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 作戦変更: 「スター」のリンクは必ずあるはずなので、そこから見つける
        # URLが '/stargazers' で終わるリンクをすべて探す
        star_links = soup.find_all('a', href=lambda x: x and x.endswith('/stargazers'))

        if not star_links:
            print("警告: スターへのリンクすら見つかりませんでした。")
            print("debug_github.html を保存します。中身を確認してください。")
            with open("debug_github.html", "w", encoding="utf-8") as f:
                f.write(response.text)
            return

        print(f"{len(star_links)} 件のリポジトリ候補を検出しました。")

        for star_link in star_links:
            # --- 親要素をたどって情報を探す ---
            
            # スターリンクの親（コンテナ）を探す
            # 通常、liタグかdivタグの中にすべてがまとまっています
            container = star_link.find_parent('li')
            if not container:
                container = star_link.find_parent('div', class_='Box-row') # 旧形式の念の為
            if not container:
                # もし親が見つからなければ、スターリンクの3階層くらい上を仮のコンテナとする
                container = star_link.parent.parent.parent

            # 1. スター数 (取得したリンクのテキストを整形)
            stars = star_link.text.strip()
            # 数字以外（カンマなど）が入っている場合があるのでそのまま使うか、必要ならreplaceする

            # 2. リポジトリ名
            # コンテナの中にある h3 タグを探すのが確実
            h3_tag = container.find('h3')
            if h3_tag and h3_tag.find('a'):
                repo_name = h3_tag.find('a').text.strip()
            else:
                # h3がない場合、リンクのhrefが /google/ で始まるものを探す
                name_link = container.find('a', href=lambda x: x and x.startswith('/google/') and 'stargazers' not in x)
                repo_name = name_link.text.strip() if name_link else "Unknown"

            # 3. 言語
            # itemprop="programmingLanguage" を探す
            lang_tag = container.find('span', itemprop='programmingLanguage')
            if lang_tag:
                language = lang_tag.text.strip()
            else:
                # 言語タグがない場合、色丸(repo-language-color)の隣のspanを探すなどの推測もできるが、今回は"No Language"とする
                language = "No Language"

            # データを保存
            cursor.execute('''
                INSERT INTO repositories (name, language, stars)
                VALUES (?, ?, ?)
            ''', (repo_name, language, stars))
            
            print(f"保存: {repo_name} | {language} | Stars:{stars}")
            
            time.sleep(1)

        conn.commit()
        print("\n全データの保存が完了しました。\n")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

    # 3. 表示
    print("--- DB保存データの表示 ---")
    cursor.execute('SELECT * FROM repositories')
    rows = cursor.fetchall()
    
    if not rows:
        print("データがありません。")
    
    for row in rows:
        print(f"ID:{row[0]}, Name:{row[1]}, Lang:{row[2]}, Stars:{row[3]}")

    conn.close()

if __name__ == "__main__":
    main()


DB準備完了。強力な検索モードでスクレイピングを開始します...
30 件のリポジトリ候補を検出しました。
保存: skia-buildbot | No Language | Stars:158
保存: or-tools | No Language | Stars:13k
保存: angle | No Language | Stars:3.8k
保存: tunix | No Language | Stars:1.9k
保存: nomulus | No Language | Stars:1.8k
保存: device-infra | No Language | Stars:58
保存: osv-scalibr | No Language | Stars:538
保存: dwh-migration-tools | No Language | Stars:54
保存: perfetto | No Language | Stars:5k
保存: koladata | No Language | Stars:27
保存: desugar_jdk_libs | No Language | Stars:389
保存: conscrypt | No Language | Stars:1.4k
保存: tcmalloc | No Language | Stars:5k
保存: dawn | No Language | Stars:781
保存: meridian | No Language | Stars:1.2k
保存: filonov | No Language | Stars:11
保存: site-kit-wp | No Language | Stars:1.3k
保存: xls | No Language | Stars:1.4k
保存: nearby | No Language | Stars:888
保存: osv-scanner | No Language | Stars:8.1k
保存: skia | No Language | Stars:10k
保存: boringssl | No Language | Stars:2.1k
保存: brax | No Language | Stars:2.9k
保存: garf | No Language | Stars:11